In [4]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas sqlalchemy

# This is final test 

In [121]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
from sqlalchemy import create_engine
import urllib.request
import requests
import base64

client_id = "ab97038481a84b63b346d77d6d16fbf7"
client_secret = "0eff8016700e4f058f3036dde150e6a4"

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

limit = 50  # Maximum is 50

def find_by_id(data, target_id):
    return [item for item in data if item["id"] == target_id][0]

def getNepaliSongs(offset, artist):
    # Search parameters
    search_query = f'artist:{artist}'
    # Search for Nepali songs with offset
    results = sp.search(q=search_query, type='track', limit=limit, offset=offset)
    return results['tracks']['items']

def get_access_token():
    data = {'grant_type': 'client_credentials'}
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_str = f"{client_id}:{client_secret}"
    auth_bytes = auth_str.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    headers = {
        'Authorization': f'Basic {auth_b64}'
    }
    data = {'grant_type': 'client_credentials'}
    r = response = requests.post(auth_url, headers=headers, data=data)
    return r.json()['access_token']


def get_audio_features(track_id):
    audio_features = {}
    GET_TRACK_FEATURES_ENDPOINT = f'https://api.spotify.com/v1/audio-features?ids={track_id}'
    headers = {"Authorization": "Bearer " + token}
    audio_features_res = requests.get(url=GET_TRACK_FEATURES_ENDPOINT, headers=headers)
    return audio_features_res.json();


token = get_access_token()
# while(totalSongs < 10):

def seperateTrackIDByComma(items):
    ids = "";
    for track in items:
        trackID = track['id']
        if(ids==""):
            ids=trackID
        else:
            ids=ids+f',{trackID}' 
    return ids

db_username = "root"
db_password = "root"
db_host = "localhost"
db_name = "spotifydb"
engine = create_engine(f"mysql://{db_username}:{db_password}@{db_host}/{db_name}")
offset = 0;
count = 0;
while(offset < 1000):
    count = count + 1
    items = getNepaliSongs(offset, "Ram K")
    if(len(items)==0):
        offset = 500
        break
    offset = offset + limit
    ids = seperateTrackIDByComma(items)
    audio_features = get_audio_features(ids)
    dfs = []
    tracks_info = []
    for track_features in audio_features['audio_features']:
        trackDetail = find_by_id(items, track_features['id'])   
        track_info = {
            "id": track_features['id'],
            "Track Name": trackDetail['name'],
            "Artists": ', '.join([artist['name'] for artist in trackDetail['artists']]),
            "Preview URL": trackDetail['preview_url'],
            "Acousticness": track_features['acousticness'],
            "Danceability": track_features['danceability'],
            "Energy": track_features['energy'],
            "Instrumentalness": track_features['instrumentalness'],
            "Liveness": track_features['liveness'],
            "Loudness": track_features['loudness'],
            "Speechiness": track_features['speechiness'],
            "Tempo": track_features['tempo'],
            "Valence": track_features['valence']
        }
        tracks_info.append(track_info)
    dfs.append(pd.DataFrame(tracks_info))
    df = pd.concat(dfs, ignore_index=True)
    try:
        # Connect to the database
        with engine.connect() as conn:
            # Write DataFrame to MySQL
            df.to_sql(name="spotify_music", con=conn, if_exists="append", index=False)
            print(f"Data successfully stored in MySQL database. {len(tracks_info)}")
    except Exception as e:
        print("An error occurred while writing to the database:", e)
    finally:
        # Dispose of the engine
        engine.dispose()
    if(count == 4):
        time.sleep(40)
        count=0
df.head()

,id,Track Name,Artists,Preview URL,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence
0,59eEJ8LsL6RJdnRzOUtOQ7,Ramayan Chaupai,Tara Devi,https://p.scdn.co/mp3-preview/08a78c9a7587acfe...,0.260,0.552,0.596,0.0,0.169,-3.445,0.0285,143.772,0.651
1,3zL1R05MfYHClIJ1pHqQF0,Apke Charno Mein Hanumat - Hindi,Tara Devi,https://p.scdn.co/mp3-preview/8e24d5517273865f...,0.384,0.661,0.724,0.0,0.312,-6.093,0.0282,92.987,0.915
2,569ZgBccgH7D0lsWmNkeim,Tere Bin,Tara Devi,https://p.scdn.co/mp3-preview/1b2a2b73a836d5f0...,0.632,0.427,0.807,0.0,0.152,-3.389,0.0659,187.915,0.620
3,1nayAUb1pYk2PCDokuYCmA,Mera Apki Kripa Se,Tara Devi,https://p.scdn.co/mp3-preview/7184f2204f52bd06...,0.463,0.560,0.786,0.0,0.388,-5.106,0.0473,93.649,0.733
4,3vTuPvw8QpeMqJODIIL73J,Raam Dev Chalisa,Tara Devi,https://p.scdn.co/mp3-preview/bc8fd824e9839f4d...,0.492,0.354,0.912,0.0,0.407,-4.501,0.1010,83.295,0.720
